In [1]:
import xarray as xr
import numpy as np
from climatico import enso
import matplotlib.pyplot as plt
import cftime
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from climatico.util import weighted_mean, pacific_lon
import matplotlib.patches as mpatches
import cartopy.feature as cfeature
from shapely.geometry.polygon import Polygon

In [2]:
from config import directory_figs, directory_data

In [3]:
def grab_enso_spatial(ds, indices, ds_oni, year1, year2, cutoff, filevar='SST'):
    """
    Use the input seasonal file ('QS-DEC') and precomputed Nino-index to extract Nino(a) 
    event indices in time array of SST file.
    Then extract the variable associated with the Nino(a) events. These focus on DJF months.
    
    Args:
        ds (xarray data array): Seasonal variable to filter for plotting 
                                    (e.g., ds.resample(time='QS-DEC').mean(skipna=True)).
        indices (numpy array): ONI values as numpy array 
                                    (e.g., control_nino.resample(time='QS-DEC').mean(skipna=True).values).
        ds_oni (xarray data array): ONI seasonal xarray 
                                    (e.g., control_nino.resample(time='QS-DEC').mean(skipna=True)).
        year1 (int): First year for range filtering.
        year2 (int): Second year for range filtering.
        cutoff (float): Cutoff of ENSO events. E.g., +/-0.5 for ONI.
        filevar (str): Variable name for input ds. Defaults to ``SST.``
    """
    # do checks
    check1 = ds[filevar].isel(time=np.where(indices>=cutoff)[0])[(ds[filevar].isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==12)].coords['time']
    check2 = ds_oni.isel(time=np.where(indices>=cutoff)[0])[(ds_oni.isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==12)].coords['time']
    assert np.all(check1.values==check2.values), "Nino events don't match"
    print("Nino check passed")
    
    check1 = ds[filevar].isel(time=np.where(indices<=cutoff)[0])[(ds[filevar].isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==12)].coords['time']
    check2 = ds_oni.isel(time=np.where(indices<=cutoff)[0])[(ds_oni.isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==12)].coords['time']
    assert np.all(check1.values==check2.values), "Nina events don't match"
    print("Nina check passed")
    
    ### nino
    # filter for djf quarters
    sv_indices = ds[filevar].isel(time=np.where(indices>=cutoff)[0])[(ds[filevar].isel(time=np.where(indices>=cutoff)[0])['time'].dt.month==12)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['time'].dt.year>=year1)&(sv_indices['time'].dt.year<=year2)]
    # take spatial mean for plotting
    nino = sv_indices.mean(dim='time')
    print("Nino done")

    ### nina
    # filter for djf quarters
    sv_indices = ds[filevar].isel(time=np.where(indices<=cutoff)[0])[(ds[filevar].isel(time=np.where(indices<=cutoff)[0])['time'].dt.month==12)]
    # filter for correct year range
    sv_indices = sv_indices[(sv_indices['time'].dt.year>=year1)&(sv_indices['time'].dt.year<=year2)]
    # take spatial mean for plotting
    nina = sv_indices.mean(dim='time')
    print("Nina done")
    
    return nino, nina

In [4]:
# list of filenames to do this for:
# SSTs
file_g02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.SST.*.nc'
file_g04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalG04Sv.pop.h.SST.*.nc'
file_p02sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP02Sv.pop.h.SST.*.nc'
file_p04sv = 'b1d.e11.B1850LENS.f09_g16.FWAtSalP04Sv.pop.h.SST.*.nc'
file_psalt = 'b1d.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.SST.*.nc'
file_cntrl = 'b1d.e11.B1850C5CN.f09_g16.005.pop.h.SST.*.nc'
# list of filenames to do this for
TEMP_g02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG02Sv.pop.h.TEMP.*.nc'
TEMP_g04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalG04Sv.pop.h.TEMP.*.nc'
TEMP_p02sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP02Sv.pop.h.TEMP.*.nc'
TEMP_p04sv = 'b.e11.B1850LENS.f09_g16.FWAtSalP04Sv.pop.h.TEMP.*.nc'
TEMP_psalt = 'b.e11.B1850LENS.f09_g16.FWPaSalP04Sv.pop.h.TEMP.*.nc'
TEMP_cntrl = 'b.e11.B1850C5CN.f09_g16.005.pop.h.TEMP.*.nc'

In [5]:
nino = enso.DefineNino(nino='nino34', lats='lat', lons='lon', cutoff=0.5, runningmean=3)
nino_TEMP = enso.DefineNino(nino='nino34', cutoff=0.5, runningmean=3)

In [7]:
####################################################################

ds2 = xr.open_mfdataset(f'{directory_data}{file_cntrl}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds2 = ds2.isel(z_t=0)

print("indices started")
# compute indices
control_nino = nino.compute_index(ds2['SST'].groupby('time.month'), 
                                  ds2['SST'].groupby('time.month').mean(skipna=True), 
                                  linear_detrend=False, lat_name='lat')
# grab numpy array
control_index = control_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_cntrl}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)

z_t_coord = ds_['TEMP'].sel(time=slice(cftime.DatetimeNoLeap(201, 1, 1, 0, 0),cftime.DatetimeNoLeap(500, 12, 1, 0, 0))).resample(
                           time='AS').mean(skipna=True).mean(dim=['time']).mean(dim=['nlat'], skipna=True).sel(z_t=slice(500., 300. * 100.)).coords['z_t'].values

lon_coord = ds_['TEMP'].coords['TLONG'].values

controlnino, controlnina = grab_enso_spatial(
                                              ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500., 300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                              indices = control_index, 
                                              ds_oni = control_nino.resample(time='QS-DEC').mean(skipna=True),
                                              year1 = 801, 
                                              year2 = 1599,
                                              cutoff = nino.cutoff, 
                                              filevar = 'TEMP')

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 101 times more chunks
  return self.array[key]


indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


Nino check passed
Nina check passed
Nino done
Nina done


In [8]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_g02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
g02sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')
# grab numpy array
g02sv_index = g02sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_g02sv}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)

g02svnino, g02svnina = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = g02sv_index, 
                                          ds_oni = g02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 51 times more chunks
  return self.array[key]


indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = u

Nino check passed
Nina check passed
Nino done
Nina done


In [9]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_g04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
g04sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')
# grab numpy array
g04sv_index = g04sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_g04sv}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)

g04svnino, g04svnina = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = g04sv_index, 
                                          ds_oni = g04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


Nino check passed
Nina check passed
Nino done
Nina done


In [10]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_p02sv}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
p02sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')
# grab numpy array
p02sv_index = p02sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_p02sv}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)

p02svnino, p02svnina = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = p02sv_index, 
                                          ds_oni = p02sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = u

Nino check passed
Nina check passed
Nino done
Nina done


In [11]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_p04sv}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
p04sv_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')
# grab numpy array
p04sv_index = p04sv_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_p04sv}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)

p04svnino, p04svnina = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = p04sv_index, 
                                          ds_oni = p04sv_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 201, 
                                          year2 = 500, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = u

Nino check passed
Nina check passed
Nino done
Nina done


In [12]:
####################################################################

ds1 = xr.open_mfdataset(f'{directory_data}{file_psalt}',
                        combine='by_coords',
                        preprocess=nino.nino)

# reduce dims to time, lat, lon
ds1 = ds1.isel(z_t=0)

print("indices started")
# compute indices
psalt_nino = nino.compute_index(ds1['SST'].groupby('time.month'), 
                                ds2['SST'].groupby('time.month').mean(skipna=True), 
                                linear_detrend=False, lat_name='lat')
# grab numpy array
psalt_index = psalt_nino.resample(time='QS-DEC').mean(skipna=True).values
print("indices done")

#################################################################### TEMP

ds_ = xr.open_mfdataset(f'{directory_data}{TEMP_psalt}',
                        combine='by_coords',
                        preprocess=nino_TEMP.nino)

psaltnino, psaltnina = grab_enso_spatial(
                                          ds = ds_.mean(dim=['nlat'], skipna=True).sel(z_t=slice(500.,300. * 100.)).resample(time='QS-DEC').mean(skipna=True), 
                                          indices = psalt_index, 
                                          ds_oni = psalt_nino.resample(time='QS-DEC').mean(skipna=True),
                                          year1 = 101, 
                                          year2 = 250, 
                                          cutoff = nino.cutoff, 
                                          filevar = 'TEMP')

####################################################################

indices started


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/xarray/core/indexing.py:1361: PerformanceWarning: Slicing with an out-of-order index is generating 50 times more chunks
  return self.array[key]


indices done


/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/numpy/core/_methods.py:151: RuntimeWarning: invalid value encountered in reduce
  ret = u

Nino check passed
Nina check passed
Nino done
Nina done


In [13]:
#%%capture
cntrl_diff_nino = controlnino.values
g02sv_diff_nino = g02svnino.values
g04sv_diff_nino = g04svnino.values
p02sv_diff_nino = p02svnino.values
p04sv_diff_nino = p04svnino.values
psalt_diff_nino = psaltnino.values

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/m

In [14]:
data_assemble=xr.Dataset({
                 'cntrl_nino':(['z_t','a'], cntrl_diff_nino),
                 'g02sv_nino':(['z_t','a'], g02sv_diff_nino),
                 'g04sv_nino':(['z_t','a'], g04sv_diff_nino),
                 'p02sv_nino':(['z_t','a'], p02sv_diff_nino),
                 'p04sv_nino':(['z_t','a'], p04sv_diff_nino),
                 'psalt_nino':(['z_t','a'], psalt_diff_nino),
                 'TLONG':(['nlat','nlon'],  lon_coord)},
                 coords = {'z_t':(['z_t'],  z_t_coord)})

In [15]:
data_assemble.to_netcdf(f'{directory_data}ensoslabs_ninodata.nc')

In [16]:
#%%capture
cntrl_diff_nina = controlnina.values
g02sv_diff_nina = g02svnina.values
g04sv_diff_nina = g04svnina.values
p02sv_diff_nina = p02svnina.values
p04sv_diff_nina = p04svnina.values
psalt_diff_nina = psaltnina.values

/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/glade/work/molina/m

In [17]:
data_assemble=xr.Dataset({
                 'cntrl_nina':(['z_t','a'], cntrl_diff_nina),
                 'g02sv_nina':(['z_t','a'], g02sv_diff_nina),
                 'g04sv_nina':(['z_t','a'], g04sv_diff_nina),
                 'p02sv_nina':(['z_t','a'], p02sv_diff_nina),
                 'p04sv_nina':(['z_t','a'], p04sv_diff_nina),
                 'psalt_nina':(['z_t','a'], psalt_diff_nina),
                 'TLONG':(['nlat','nlon'],  lon_coord)},
                 coords = {'z_t':(['z_t'],  z_t_coord)})

In [18]:
data_assemble.to_netcdf(f'{directory_data}ensoslabs_ninadata.nc')